In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import os
from resnet_module import ResNetModel
import torchmetrics.classification

from resnet_training import get_predictions, calculate_gender_bias, calculate_age_bias, calculate_hairiness_bias, calculate_skin_tone_bias

In [ ]:
debiasing_runs = ["folm5wbd", "00riantl", "uiznsjjj", "30y8q6vf", "hi6ej7fl", "nl10rh1v", "xfh2oa3v", "i8zwc0lj", "062eb1li", "79uu4gax"]
no_debiasing_runs = ["kzozqfbw", "kdt4bf3x", "k7w8ngzr", "w572bz9r", "9e0j4szk", "3x8qz40u", "n3w92l9t", "ylc3xokv", "w2fevh40", "5guuqk3t"]

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
num_classes=2
metric = torchmetrics.classification.BinaryAccuracy()

results = {}

for run in debiasing_runs.extend(no_debiasing_runs):
    results[run] = {}
    checkpoint_dir = f'./bias-skin-lesion-detection/{run}/checkpoints/'
    file_name = os.listdir(checkpoint_dir)[0]
    model_path = os.path.join(checkpoint_dir, file_name)
    
    model = ResNetModel.load_from_checkpoint(model_path, map_location=device)
    predictions, all_labels = get_predictions(model)

    gender_bias = calculate_gender_bias(predictions, all_labels, metric, num_classes=2)
    results[run].update(gender_bias)
    age_bias = calculate_age_bias(predictions, all_labels)
    results[run].update(age_bias)

    hairiness_bias = calculate_hairiness_bias(predictions, all_labels)
    results[run].update(hairiness_bias)

    skin_tone_bias = calculate_skin_tone_bias(predictions, all_labels)
    results[run].update(skin_tone_bias)

In [ ]:
print(results)